In [29]:
!pip install -q pip --upgrade
!pip install -q wrapt --upgrade --ignore-installed
!pip install -q transformers==2.8.0

ERROR: awscli 1.18.49 has requirement rsa<=3.5.0,>=3.1.2, but you'll have rsa 4.0 which is incompatible.
ERROR: Could not install packages due to an EnvironmentError: [Errno 28] No space left on device



In [ ]:
!pip install -q tensorflow==2.1.0 --upgrade --ignore-installed

ERROR: awscli 1.18.49 has requirement rsa<=3.5.0,>=3.1.2, but you'll have rsa 4.0 which is incompatible.


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import functools
import multiprocessing

import pandas as pd
from datetime import datetime
import subprocess
import sys
import tensorflow as tf
from transformers import DistilBertTokenizer
from tensorflow import keras
import os
import re
import collections
import argparse
import json
import os
import pandas as pd
import csv
import glob
from pathlib import Path

# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
DATA_COLUMN = 'review_body'
LABEL_COLUMN = 'star_rating'
LABEL_VALUES = [1, 2, 3, 4, 5]

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
tokenizer.tokenize("This here's an example of using the BERT tokenizer")


class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self,
               input_ids,
               input_mask,
               segment_ids,
               label_id):
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.segment_ids = segment_ids
    self.label_id = label_id
    
    
class Input(object):
  """A single training/test input for sequence classification."""

  def __init__(self, text, label=None):
    """Constructs an Input.
    Args:
      text: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
    self.text = text
    self.label = label
    
    
def convert_input(text_input):
  """Converts a single `Input` into a single `InputFeatures`."""

  label_map = {}
  for (i, label) in enumerate(LABEL_VALUES):
    label_map[label] = i

  # Next, we need to preprocess our data so that it matches the data BERT was trained on. For this, we'll need to do a couple of things (but don't worry--this is also included in the Python library):
  # 
  # 
  # 1. Lowercase our text (if we're using a BERT lowercase model)
  # 2. Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"])
  # 3. Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
  # 4. Map our words to indexes using a vocab file that BERT provides
  # 5. Add special "CLS" and "SEP" tokens (see the [readme](https://github.com/google-research/bert))
  # 6. Append "index" and "segment" tokens to each input (see the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf))
  # 
  # We don't have to worry about these details.  The Transformers tokenizer does this for us.
  # 
  tokens = tokenizer.tokenize(text_input.text)
  print(tokens)

  # Account for [CLS] and [SEP] with "- 2"
  if len(tokens) > MAX_SEQ_LENGTH - 2:
    tokens = tokens[0:(MAX_SEQ_LENGTH - 2)]

  # The convention in BERT is:
  # (a) For sequence pairs:
  #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
  #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
  # (b) For single sequences:
  #  tokens:   [CLS] the dog is hairy . [SEP]
  #  type_ids: 0     0   0   0  0     0 0
  #
  # Where "type_ids" are used to indicate whether this is the first
  # sequence or the second sequence. The embedding vectors for `type=0` and
  # `type=1` were learned during pre-training and are added to the wordpiece
  # embedding vector (and position vector). This is not *strictly* necessary
  # since the [SEP] token unambiguously separates the sequences, but it makes
  # it easier for the model to learn the concept of sequences.
  #
  # For classification tasks, the first vector (corresponding to [CLS]) is
  # used as the "sentence vector". Note that this only makes sense because
  # the entire model is fine-tuned.
  all_tokens = []
  segment_ids = []
  all_tokens.append("[CLS]")
  segment_ids.append(0)
  for token in tokens:
    all_tokens.append(token)
    segment_ids.append(0)
  all_tokens.append("[SEP]")
  segment_ids.append(0)

  input_ids = tokenizer.convert_tokens_to_ids(all_tokens)

  # The mask has 1 for real tokens and 0 for padding tokens. Only real
  # tokens are attended to.
  input_mask = [1] * len(input_ids)

  # Zero-pad up to the sequence length.
  while len(input_ids) < MAX_SEQ_LENGTH:
    input_ids.append(0)
    input_mask.append(0)
    segment_ids.append(0)

  assert len(input_ids) == MAX_SEQ_LENGTH
  assert len(input_mask) == MAX_SEQ_LENGTH
  assert len(segment_ids) == MAX_SEQ_LENGTH

  label_id = label_map[text_input.label]

  feature = InputFeatures(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids,
      label_id=label_id)

  print('input_ids\n{}'.format(feature.input_ids))
  print('input_mask\n{}'.format(feature.input_mask))
  print('segment_ids\n{}'.format(feature.segment_ids))
  print('label_id\n{}'.format(feature.label_id))

  return feature


# We'll need to transform our data into a format BERT understands. This involves two steps. 
# First, we create  `InputExample`'s using the constructor provided in the BERT library.
# - `text` is the text we want to classify, which in this case, is the `Request` field in our Dataframe. 
# - `label` is the label for our example (0 or 1)
def transform_df_to_tfrecord(df):
    # Use the InputExample class from BERT's run_classifier code to create examples from the data
    inputs = df.apply(lambda x: Input(text = x[DATA_COLUMN], 
                                      label = x[LABEL_COLUMN]), 
                      axis = 1)

    # Convert our inputs to TFRecords (.tfrecord protobuf) that works with BERT and TensorFlow.

    tf_records = []
    for (input_idx, text_input) in enumerate(inputs):
      if input_idx % 10000 == 0:
          print("Writing input %d of %d" % (input_idx, len(inputs)))

      feature = convert_input(text_input)

      features = collections.OrderedDict()
      features["input_ids"] = tf.train.Feature(int64_list=tf.train.Int64List(value=feature.input_ids))
      features["input_mask"] = tf.train.Feature(int64_list=tf.train.Int64List(value=feature.input_mask))
      features["segment_ids"] = tf.train.Feature(int64_list=tf.train.Int64List(value=feature.segment_ids))
      features["label_ids"] = tf.train.Feature(int64_list=tf.train.Int64List(value=[feature.label_id]))

      tf_record = tf.train.Example(features=tf.train.Features(feature=features))
      tf_records.append(tf_record.SerializeToString())

    return tf_records


In [23]:
data = [
        ['This is a great review.', 5],
#        ['This is a bad review.', 1],
       ]

df = pd.DataFrame(data, columns=['review_body', 'star_rating'])



In [28]:
tf.version

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-28-0157324439fb>", line 1, in <module>
    tf.version
AttributeError: module 'tensorflow' has no attribute 'version'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 1863, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'AttributeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/tensorflow_core/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_te

AttributeError: module 'tensorflow' has no attribute 'version'

In [24]:
tf_records = transform_df_to_tfrecord(df)
tf_records

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Writing input 0 of 1
['this', 'is', 'a', 'great', 'review', '.']
input_ids
[101, 2023, 2003, 1037, 2307, 3319, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
input_mask
[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
segment_ids
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

AttributeError: module 'tensorflow' has no attribute 'train'